# Testing Individual components of the FV HE scheme

In [1]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'



## Keygeneration

In [2]:
poly_modulus = 128
bit_sizes= [40, 40, 40, 40]
plain_modulus = 64
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [3]:
# print(len(sk.data))
print('secret key values : ', sk.data)

secret key values :  [[1099511600896, 0, 1, 0, 1, 0, 1099511600896, 1, 1, 1, 0, 0, 0, 0, 1099511600896, 0, 1099511600896, 1099511600896, 1099511600896, 0, 1099511600896, 0, 1099511600896, 1099511600896, 1, 0, 1099511600896, 1099511600896, 0, 1, 1, 1099511600896, 1, 1, 1, 1099511600896, 0, 0, 1, 1099511600896, 1099511600896, 1, 1099511600896, 0, 0, 0, 1099511600896, 0, 1099511600896, 0, 1, 1, 1, 1099511600896, 1, 0, 0, 0, 0, 1, 1099511600896, 1099511600896, 1, 1, 1, 1099511600896, 1, 0, 1099511600896, 1, 1, 1, 0, 0, 1, 0, 1099511600896, 1, 0, 0, 1, 0, 0, 1099511600896, 0, 0, 1, 1, 0, 1099511600896, 1, 1099511600896, 1, 1099511600896, 1099511600896, 0, 0, 1, 0, 0, 1, 1, 0, 1099511600896, 1099511600896, 1099511600896, 1, 1099511600896, 1099511600896, 0, 0, 1, 1099511600896, 1, 0, 0, 0, 1099511600896, 0, 1099511600896, 1099511600896, 1099511600896, 1, 1, 1099511600896, 0, 1099511600896, 1], [1099511603712, 0, 1, 0, 1, 0, 1099511603712, 1, 1, 1, 0, 0, 0, 0, 1099511603712, 0, 1099511603712, 

In [4]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [5]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,10)
ri2 = random.randint(0,10)
ri3 = random.randint(0,10)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[0, 0, 0, 1]     [0, 1, 1]     [1, 1]


### Decodes back to Integer

In [6]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

8
6
3


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [7]:
encrypter = Encryptor(ctx, pk)

In [8]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [9]:
decrypter = Decryptor(ctx, sk)

In [10]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [11]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

8     6     3


## Evaluator

In [12]:
eval = Evaluator(ctx)

In [13]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

14


In [14]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

14


In [15]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

14


### Verify result

In [16]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [17]:
prod = eval._mul_cipher_cipher(ct1, ct2)
print("\n\nct1 :",ct1.data)
print("\n\nct2 :",ct2.data)
print('\n\n')

result = decrypter.decrypt(prod)
result = int_encoder.decode(result)

print('final result: ', result)



ct1 : [[[482845931020, 1089247671256, 299861919792, 798431905758, 482003085589, 377533010930, 571263229375, 674339215757, 559982597070, 135346064966, 772313616196, 146151457107, 812258975817, 990324777426, 506795713871, 116740413067, 154275050420, 66941523721, 806357379230, 999482502986, 762184305432, 1019342834382, 888915964232, 192114052333, 675303423699, 917872342190, 544333023887, 752073319048, 198371818848, 521911928441, 92664775830, 578620787462, 1083084527218, 467364460271, 109753929368, 571480321104, 1050804218655, 912173200171, 1042512367669, 619486889872, 623043715959, 897785660228, 611061359907, 934775374745, 743546886022, 1037374839693, 229013322711, 269278343343, 658059837316, 584093583696, 500370304856, 1012875056880, 896942386925, 773511464353, 414499457806, 336764678262, 988990652131, 751500658903, 978178594963, 139100393031, 861363443404, 481763403909, 1010962727843, 110327347538, 831000985594, 188068624505, 742050374025, 526706978005, 306803983470, 885183935634, 528

In [18]:
print(ri1 * ri2, "    ", result)
assert ri1 * ri2 == result

48      48


# Relin operation

In [19]:
print(len(prod.data))
print(prod.data)

3
[[[156779049204, 1095298045911, 733971107231, 754699892198, 33890480832, 607803675996, 1036747470958, 974764296932, 329060562378, 542808924107, 1054951947798, 735635449166, 917590027482, 438676555234, 1018236287139, 76761873485, 730613711760, 597474131891, 440603444288, 475086376772, 709146235523, 459922546453, 330852140497, 167881476150, 174932713903, 474132651016, 576139964833, 137385263667, 211850692393, 808405903627, 87630995197, 799740118991, 166587925707, 1099189411326, 290518491960, 998228750724, 67613899429, 338982170736, 687837044729, 118840657068, 21111190898, 784906255579, 263129170491, 477637461013, 760124155989, 186017763467, 169018056269, 332070332252, 941182127701, 448700342413, 258908339771, 461479570403, 13471065865, 324447126594, 801494633751, 494963542483, 509772482231, 853285209917, 680004865950, 586316131976, 1056604841209, 84393629528, 212611101267, 959180991497, 232108034055, 509205718641, 289804206157, 672261976080, 85606261691, 245055794, 36280292235, 3266042

In [20]:
relin_key = keygenerator.get_relin_keys()
print(relin_key)

[[[[456805368686, 166669143374, 146139678590, 936047233290, 765721017929, 356345455513, 787798813368, 554707087756, 914117103968, 213965638442, 515522951609, 332262632877, 982300869695, 1005732856491, 196016382691, 179794975231, 513292892758, 324277352390, 230867819640, 886677199339, 720862006954, 729160757089, 1052844712130, 1042819547497, 557126672335, 77235358527, 836704779820, 92512873154, 850865110769, 681148035400, 1047589700656, 440459581761, 201021397932, 1039683852882, 399379297586, 1015614886592, 1046776569977, 731474408895, 374669947930, 700712739525, 474581804021, 619028253117, 990950874056, 130570383888, 661271788886, 428988769292, 115633373902, 871082061114, 137906207379, 1011186884946, 281137181977, 594939720473, 528886848573, 860211071618, 4748255994, 425726117967, 423216517404, 1018390597991, 698448235612, 318186543679, 342320862938, 385152633662, 1077908765436, 740527263123, 623781128567, 54360700136, 784454165011, 234388805198, 285504740077, 515903477827, 76343446638

In [21]:
print(len(prod.data))
relin_prod = eval.relin(prod, relin_key)
print(len(relin_prod.data),'\n\n')
print(relin_prod.data)

3
2
[[[680520965236, 481909540550, 75803201411, 303382282679, 227465218996, 221492565326, 257343996473, 89973895450, 986761136438, 158995613636, 553373130177, 428896191311, 184197416800, 725177512568, 925109727319, 390393745067, 576660033676, 368411785574, 192216464810, 553947015106, 272155010123, 4100955798, 700534292373, 995641823751, 1035304429119, 229531523820, 287776826810, 1034053197056, 711700596687, 899382278559, 183463498765, 907603883175, 1006000483737, 700267530271, 625162098525, 641985863243, 43180792208, 383268399880, 264374160421, 289831009302, 1031277796710, 185378151330, 557299175462, 11321938251, 887952832044, 509850922483, 1056962528133, 189046508960, 409084935932, 1030601997082, 571949174722, 878862073498, 168804032815, 1055378705420, 735063405068, 475502238541, 588121960875, 320170072431, 1056281754670, 125222553822, 601078814632, 110647524382, 645559026702, 644011700633, 190530353848, 101399390895, 667702586839, 1024268616322, 446147063745, 527265732948, 6958611615

In [22]:
print(int_encoder.decode(decrypter.decrypt(relin_prod)), "   ", int_encoder.decode(decrypter.decrypt(prod)))
assert int_encoder.decode(decrypter.decrypt(relin_prod)) == int_encoder.decode(decrypter.decrypt(prod))

1179429619693278859725548976679661557174     48


AssertionError: 